In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
torch.manual_seed(1)

In [0]:
x_data = [[1, 2], [2, 3], [3, 1], [4, 3], [5, 3], [6, 2]]
y_data = [[0], [0], [0], [1], [1], [1]]

In [0]:
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

In [5]:
print(x_train.shape)
print(y_train.shape)

torch.Size([6, 2])
torch.Size([6, 1])


In [6]:
print("e^1 equals : ", torch.exp(torch.FloatTensor([1])))

e^1 equals :  tensor([2.7183])


In [0]:
W = torch.zeros([2, 1], requires_grad=True)
b = torch.zeros(1, requires_grad=True)

In [0]:
hypothesis = 1 / (1 + torch.exp(-(x_train.matmul(W) + b)))

In [9]:
print(hypothesis)
print(hypothesis.shape)

tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000]], grad_fn=<MulBackward0>)
torch.Size([6, 1])


In [10]:
print("1 / (1 + e^{-1}) equals: ", torch.sigmoid(torch.FloatTensor([1])))

1 / (1 + e^{-1}) equals:  tensor([0.7311])


In [0]:
hypothesis = torch.sigmoid(x_train.matmul(W) + b)

In [12]:
print(hypothesis)
print(hypothesis.shape)

tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000]], grad_fn=<SigmoidBackward>)
torch.Size([6, 1])


In [13]:
losses = -(y_train * torch.log(hypothesis[0]) + (1 - y_train) * torch.log(1 - hypothesis))
print(losses)

tensor([[0.6931],
        [0.6931],
        [0.6931],
        [0.6931],
        [0.6931],
        [0.6931]], grad_fn=<NegBackward>)


In [14]:
cost = losses.mean()
print(cost)

tensor(0.6931, grad_fn=<MeanBackward0>)


# Computing the Cost Function with F.binary_cross_entropy

In [15]:
F.binary_cross_entropy(hypothesis, y_train)

tensor(0.6931, grad_fn=<BinaryCrossEntropyBackward>)

# Training with Low-level Binary Cross Entropy Loss

In [0]:
import numpy as np

In [34]:
from google.colab import files
uploaded = files.upload()

Saving data-03-diabetes.csv to data-03-diabetes.csv


In [0]:
xy = np.loadtxt("data-03-diabetes.csv", delimiter=',', dtype=np.float32)
x_data = xy[:, 0:-1]
y_data = xy[:, [-1]]
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

In [38]:
# 모델 초기화
W = torch.zeros([8, 1], requires_grad=True)
b = torch.zeros(1, requires_grad=True)

# optimizer 설정
optimizer = optim.SGD([W, b], lr = 1)

nb_epochs = 1000

for epoch in range(nb_epochs + 1):

  hypothesis = torch.sigmoid(x_train.matmul(W) + b)
  cost = -(y_train * torch.log(hypothesis) + (1 - y_train) * torch.log(1 - hypothesis)).mean()

  # cost로 H(x) 개선
  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  # 100번마다 로그 출력
  if epoch % 100 == 0:
    print("Epoch {:4d}/{} Cost : {:.6f}".format(epoch, nb_epochs, cost.item()))

Epoch    0/1000 Cost : 0.693148
Epoch  100/1000 Cost : 0.480661
Epoch  200/1000 Cost : 0.473846
Epoch  300/1000 Cost : 0.472511
Epoch  400/1000 Cost : 0.472051
Epoch  500/1000 Cost : 0.471851
Epoch  600/1000 Cost : 0.471758
Epoch  700/1000 Cost : 0.471713
Epoch  800/1000 Cost : 0.471692
Epoch  900/1000 Cost : 0.471681
Epoch 1000/1000 Cost : 0.471676


In [40]:
hypothesis = torch.sigmoid(x_train.matmul(W) + b)
print(hypothesis[:5])

tensor([[0.3555],
        [0.9564],
        [0.1934],
        [0.9623],
        [0.0706]], grad_fn=<SliceBackward>)


In [41]:
prediction = hypothesis >= torch.FloatTensor([0.5])
print(prediction[:5])

tensor([[False],
        [ True],
        [False],
        [ True],
        [False]])


In [42]:
print(y_train[:5])

tensor([[0.],
        [1.],
        [0.],
        [1.],
        [0.]])


In [44]:
correct_predition = prediction.float() == y_train
print(correct_predition[:5])

tensor([[True],
        [True],
        [True],
        [True],
        [True]])


In [0]:
class BinaryClassifier(nn.Module):
  def __init__(self):
    super().__init__()
    self.linear = nn.Linear(8, 1)
    self.sigmoid = nn.Sigmoid()
  
  def forward(self, x):
    return self.sigmoid(self.linear(x))

In [0]:
model = BinaryClassifier()

In [47]:
optimizer = optim.SGD(model.parameters(), lr = 1)

nb_epochs = 100
for epoch in range(nb_epochs + 1):
  hypothesis = model(x_train)

  cost = F.binary_cross_entropy(hypothesis, y_train)

  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  if epoch % 10 == 0:
    prediction = hypothesis >= torch.FloatTensor([0.5])
    correct_predition = prediction.float() == y_train
    accuracy = correct_predition.sum().item() / len(correct_predition)
    print("Epoch {:4d}/{} Cost : {:.6f} Accuracy : {:2.2f}%".format(epoch, nb_epochs, cost.item(), accuracy * 100))

Epoch    0/100 Cost : 0.691402 Accuracy : 55.47%
Epoch   10/100 Cost : 0.576109 Accuracy : 66.93%
Epoch   20/100 Cost : 0.541669 Accuracy : 72.33%
Epoch   30/100 Cost : 0.521164 Accuracy : 76.28%
Epoch   40/100 Cost : 0.508067 Accuracy : 76.55%
Epoch   50/100 Cost : 0.499238 Accuracy : 77.08%
Epoch   60/100 Cost : 0.493040 Accuracy : 77.08%
Epoch   70/100 Cost : 0.488553 Accuracy : 77.08%
Epoch   80/100 Cost : 0.485224 Accuracy : 76.42%
Epoch   90/100 Cost : 0.482704 Accuracy : 76.42%
Epoch  100/100 Cost : 0.480765 Accuracy : 76.81%
